<a href="https://colab.research.google.com/github/TonyNewness/Product-Recommendation/blob/main/Copy_of_Main_Group_2_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install implicit

In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
from implicit.als import AlternatingLeastSquares
from google.colab import files
from scipy.sparse import coo_matrix, csr_matrix

# from google.colab import drive
# drive.mount ('/content/drive')

In [ ]:
data= pd.read_csv('/content/my_dataframe.csv')

In [ ]:
data.head()

,Customer ID,Purchase Date,Product Price,Quantity,Total Purchase Amount,Payment Method,Customer Name,Age,Gender,Churn,Item Purchased
0,44605,3/5/2023 21:30,177,0,2427,PayPal,John Rivera,31,Female,0,Blouse
1,44605,16/05/2021 13:57,174,2,2448,PayPal,John Rivera,31,Female,0,Sweater
2,44605,13/07/2020 06:16,413,0,2345,Credit Card,John Rivera,31,Female,0,Jeans
3,44605,17/01/2023 13:14,396,3,937,Cash,John Rivera,31,Female,0,Sandals
4,44605,1/5/2021 11:29,259,3,2598,PayPal,John Rivera,31,Female,0,Blouse


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   Customer ID            500000 non-null  int64 
 1   Purchase Date          500000 non-null  object
 2   Product Price          500000 non-null  int64 
 3   Quantity               500000 non-null  int64 
 4   Total Purchase Amount  500000 non-null  int64 
 5   Payment Method         500000 non-null  object
 6   Customer Name          500000 non-null  object
 7   Age                    500000 non-null  int64 
 8   Gender                 500000 non-null  object
 9   Churn                  500000 non-null  int64 
 10  Item Purchased         500000 non-null  object
dtypes: int64(6), object(5)
memory usage: 42.0+ MB


In [ ]:
data.drop(columns=['Churn','Customer Name'],axis=1, inplace=True)
data

,Customer ID,Purchase Date,Product Price,Quantity,Total Purchase Amount,Payment Method,Age,Gender,Item Purchased
0,44605,3/5/2023 21:30,177,0,2427,PayPal,31,Female,Blouse
1,44605,16/05/2021 13:57,174,2,2448,PayPal,31,Female,Sweater
2,44605,13/07/2020 06:16,413,0,2345,Credit Card,31,Female,Jeans
3,44605,17/01/2023 13:14,396,3,937,Cash,31,Female,Sandals
4,44605,1/5/2021 11:29,259,3,2598,PayPal,31,Female,Blouse
...,...,...,...,...,...,...,...,...,...
499995,33308,8/10/2023 13:39,279,1,2187,PayPal,55,Male,Sneakers
499996,48835,11/23/2021 1:30,27,0,3615,Credit Card,42,Female,Dress
499997,21019,7/2/2020 14:04,17,5,2466,Cash,41,Male,Belt
499998,49234,12/30/2020 2:02,398,2,3668,Crypto,34,Female,Boots


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   Customer ID            500000 non-null  int64 
 1   Purchase Date          500000 non-null  object
 2   Product Price          500000 non-null  int64 
 3   Quantity               500000 non-null  int64 
 4   Total Purchase Amount  500000 non-null  int64 
 5   Payment Method         500000 non-null  object
 6   Age                    500000 non-null  int64 
 7   Gender                 500000 non-null  object
 8   Item Purchased         500000 non-null  object
dtypes: int64(5), object(4)
memory usage: 34.3+ MB


**Number of products that was not returened?**

In [ ]:
500000 - 405022

94978

**Number of Items sold in the store?**

In [ ]:
data['Item Purchased'].nunique()

25

In [ ]:
data['Customer ID'].nunique()

49998

In [ ]:
# Function to find the total number of each product sold
def most_Sold_Product(data):
  product_Sum = {} # declared empty dictionary to store the total sum of each product
  # The below checks if a product is in the dataframe column 'Item Purchased' exists and finds all it's occurrences in the 'Quantity' column and sum them up
  for name_Of_Product in data['Item Purchased'].unique(): # for every unique product in the column Item Purchased
    total_prod = data[data['Item Purchased'] == name_Of_Product]['Quantity'].sum() # Filter based on the product name and sum all the quantities (in the Quantity column) of the filtered product.
    product_Sum[name_Of_Product] = total_prod #assigns the sum of each product to the empty dictionary created earlier

  #sort the products based on most sold to the least sold
  sorted_product_Sum = sorted(product_Sum.items(), key=lambda x: x[1], reverse=True)

  # Print sum total of products on the dataset
  for name_Of_Product, total_prod in sorted_product_Sum:
    print('Total', name_Of_Product, 'sold:', total_prod)

most_Sold_Product(data)

Total Jewelry sold: 57142
Total Pants sold: 56817
Total Blouse sold: 56428
Total Shirt sold: 56202
Total Dress sold: 55541
Total Sweater sold: 54494
Total Jacket sold: 54258
Total Coat sold: 54020
Total Belt sold: 53816
Total Sunglasses sold: 53443
Total Sandals sold: 53210
Total Socks sold: 52799
Total Skirt sold: 52697
Total Scarf sold: 52545
Total Shorts sold: 52286
Total Hat sold: 51293
Total Handbag sold: 50855
Total Hoodie sold: 50246
Total Shoes sold: 49979
Total T-shirt sold: 48930
Total Sneakers sold: 48520
Total Backpack sold: 47625
Total Boots sold: 47603
Total Gloves sold: 46630
Total Jeans sold: 41334


In [ ]:
# Listing all unique years on the dataset
data['Purchase Date'].apply(lambda x: x.split('/')[2].split(' ')[0]).unique()

array(['2023', '2021', '2020', '2022'], dtype=object)

**How much revenue was made each year?
Sort the years by product price.**

In [ ]:
# Total purchase by Year
By_Year = data.groupby(data['Purchase Date'].apply(lambda x: x.split('/')[2].split(' ')[0]))['Total Purchase Amount'].sum()
By_Year

Purchase Date
2020    370153678
2021    366515884
2022    367270229
2023    258749191
Name: Total Purchase Amount, dtype: int64

In [ ]:
# Total Purchase for the 4 years
By_Year.sum()

1362688982

**Average Price spent per age group**

In [ ]:
# Average of Total purchase by Age
data.groupby('Age')['Total Purchase Amount'].mean().sort_values(ascending= False).head(10)

Age
68    2856.231787
67    2856.138489
66    2853.314596
70    2849.363342
63    2842.010849
65    2836.041856
64    2829.500776
69    2823.413374
59    2816.196087
61    2813.011063
Name: Total Purchase Amount, dtype: float64

**Total expenses by age**

In [ ]:
# Total purchase by Age
data.groupby('Age')['Total Purchase Amount'].sum().sort_values(ascending= False).head(10)

Age
64    27330148
70    27313997
68    27208464
51    26966361
67    26810572
69    26726431
63    26720586
61    26698288
56    26670164
43    26550621
Name: Total Purchase Amount, dtype: int64

In [ ]:
# Finding Total purchase per gender
by_gender = data.groupby('Gender')['Total Purchase Amount'].sum().sort_values(ascending= False)
by_gender

Gender
Male      681667105
Female    681021877
Name: Total Purchase Amount, dtype: int64

In [ ]:
# Finding the Average expenses by gender
print('================================================''\n Average Total purchase amount between the 2 gender''\n==============================================')
Avg_by_gender = data.groupby('Gender')['Total Purchase Amount'].mean().sort_values(ascending= False)
print(Avg_by_gender)
print('================================================''\n Average Difference between the 2 gender''\n==============================================')
Avg_by_gender.diff() # The difference of the Averages

 Average Total purchase amount between the 2 gender
Gender
Male      2725.403833
Female    2725.352071
Name: Total Purchase Amount, dtype: float64
 Average Difference between the 2 gender


Gender
Male           NaN
Female   -0.051761
Name: Total Purchase Amount, dtype: float64

In [ ]:
by_gender.sum() # Sum of the total expenses of both gender

1362688982

In [ ]:
by_gender.diff() # Difference in the total expenses of both gender

Gender
Male           NaN
Female   -645228.0
Name: Total Purchase Amount, dtype: float64

In [ ]:
# Average age for both genders
data.groupby('Gender')['Age'].mean()

Gender
Female    43.874930
Male      43.863879
Name: Age, dtype: float64

To develope a recommendation model without explicit customer ratings, we would use the **Matrix Factorization** techniques **Alternating Least Squares (ALS)**. This is because our data lacks direct feedback, we rely on implicit signals such as product quantities purchased. ALS helps uncover such implicit feedbacks patterns in customer interactions with products.

In [ ]:
data.drop(columns=['Purchase Date','Product Price','Total Purchase Amount','Payment Method','Age','Gender'],axis=1, inplace=True)
data

,Customer ID,Quantity,Item Purchased
0,44605,0,Blouse
1,44605,2,Sweater
2,44605,0,Jeans
3,44605,3,Sandals
4,44605,3,Blouse
...,...,...,...
499995,33308,1,Sneakers
499996,48835,0,Dress
499997,21019,5,Belt
499998,49234,2,Boots


In [ ]:
#Mapping Customer ID and Item Purcahsed to unique indices
data['customer_ids'] = data['Customer ID'].astype('category').cat.codes
data['item_ids'] = data['Item Purchased'].astype('category').cat.codes

In [ ]:
pivot = pd.pivot_table(data, values= 'Quantity', index= 'customer_ids', columns= 'item_ids', aggfunc= 'sum', fill_value= 0)

In [ ]:
# Converting to sparse matrix format for efficient computation, as sparse matrix holds only none zero values and as we can see above not all customer purchased all products hence we have a lot of zeros.
coo = sp.coo_matrix(pivot.values) # Coordinate format or COO format (coo_matrix()) is a scipy function that handles sparse matrix.
csr = coo.tocsr() # we convert to csr format because ALS model only works with csr data
csr
print(type(csr))

<class 'scipy.sparse._csr.csr_matrix'>


In [ ]:
customer_id_map = data[['Customer ID', 'customer_ids']].set_index('customer_ids', inplace=True)
item_id_map = data[['Item Purchased', 'item_ids']].set_index('item_ids', inplace=True)

In [ ]:
data.head(20)

,Customer ID,Quantity,Item Purchased,customer_ids,item_ids
0,44605,0,Blouse,44602,2
1,44605,2,Sweater,44602,23
2,44605,0,Jeans,44602,11
3,44605,3,Sandals,44602,14
4,44605,3,Blouse,44602,2
5,13738,2,Sneakers,13736,20
6,13738,1,Shirt,13736,16
7,13738,4,Shorts,13736,18
8,13738,2,Coat,13736,4
9,13738,4,Handbag,13736,7


In [ ]:
# # Creating the user and item interaction matrix
# user_item_matrix = data.pivot_table(index='customer_ids', columns='Item Purchased', values='Quantity', aggfunc='sum', fill_value=0) # The rows of the table(matrix) created will be filled with 'Customer ID' from our dataframe, while the columns are the diff. products and the values are the quantity sum (aggfunc='sum') of each product purchased by a customer while products that wasn't bought by a customer will be filled with value 0 (fill_value=0).
# user_item_matrix

**Model creation and Training**

In [ ]:
# Apply matrix factorization (ALS)
'''

'''
als_model = AlternatingLeastSquares(factors=10, regularization=0.1, iterations=20) # Initialise the ALS model
als_model.fit(csr.T)  # Train the ALS model with the data.

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.019958019256591797 seconds
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

**TESTING THE MODEL**

In [ ]:
# test_data['Customer ID'].count()

In [ ]:
# # Print the internal components of the CSR matrix
# print("\nCSR Matrix internal components:")
# print("data:", user_item_matrix.data)
# print("indices:", user_item_matrix.indices)
# print("indptr:", user_item_matrix.indptr)

In [ ]:
# The recommedation function
def recommend_for_customers(customer_ids, als_model, csr_matrix, number_of_recommedations = 5):

  internal_customer_id = np.where(customer_id_map == customer_ids)[0][0]
  user_interactions = csr_matrix[internal_customer_id]

  # Reshape user_interactions to ensure it's a 2D array with one row
  user_interactions = user_interactions.reshape(1, -1)

  recommendations = als_model.recommend(internal_customer_id, user_interactions, N=number_of_recommedations)
  recommended_item_ids = [item_id_map[i] for i, _ in recommendations]

  return recommended_item_ids

# Generate and print recommendations for each customer
unique_customer_ids = data['Customer ID'].unique()
for customer_ids in unique_customer_ids:
  recommendations = recommend_for_customers(customer_ids, als_model, csr, number_of_recommedations=5)
  print(f"Recommendations for customer {customer_ids}: {recommendations}")

<ipython-input-70-2c318896e279>:14: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  internal_customer_id = np.where(customer_id_map == customer_ids)[0][0]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# user_recommendations = {}

# for customer_id in data['Customer ID'].unique():
#     # Generate recommendations for the current customer
#     recommended_items = als_model.recommend(customer_id, user_item_matrix, N=5)

#     # Extract recommended products from the recommendations
#     recommended_products = [item_id for item_id, score in recommended_items]

#     # Store the recommendations for the current customer
#     user_recommendations[customer_id] = recommended_products

**EVALUATING THE MODEL**